# Иерархия холдинга
Пример вывода иерархии холдинга

## Описание задания

Задача отобразить общую аудиторию холдинга и все медиа-объекты, которые входят в указанные холдинги. 
Можно строить иерархию для всех уровней дерева в рамках одной ветки, т.е. возможно выбрать Brand и все Position, которые к нему относятся. 

Иерархия:

- Holding-Project-Section-Subsection
- Network-Network Section-Network subsection
- AdAgency-Brand-Subbrand-Position


Общие параметры:
- Период: 1 июня - 30 июня 2020 года
- География: Россия 100+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop,Web Mobile,App Online,App Offline).

список статистик для расчета:
- Unweighted Reach
- Reach


## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth", None)
# pd.set_option("display.precision", 6)
output_notebook()

Loading BokehJS ...

### Общие параметры для заданий

Для начала зададим общие параметры

In [2]:
# Задаем параметры
# выбираем тип установки mobile, т.к. в расчетах учавствуют все типы устройств (Web Desktop,Web Mobile,App Online,App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop-pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)


# задаем период расчета
date_from = '2020-06-01'
date_to = '2020-06-30'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, нас интересует города с населением 100тыс. и больше
population = rcats.get_population('Russia100+')

# задаем Возврастные группы, нас интересует 12+
ages = rcats.get_age_groups('12+')


# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:21:41
Тип установки: mobile
Начало периода: 2020-06-01
Конец периода: 2020-06-30
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [1, 2, 3]
Возрастные группы: [1, 2, 3, 4, 5, 6, 7]
Объектов в media-каталоге: 4716


### Получим ID холдинга
Для построения отчета нам необходимо получить идентификатор холдинга __Google Projects__, для этого воспользуемся блокнотом к котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Google Projects holding = 88149

## Задания
Перейдем к формированию заданий

### Задание №1 Получим иерархию проекта


In [3]:
project_name = 'hierarchy'
# Задаем медиа фильтр, в нашем случае это ID проекта Facebook
media_filter = "holding = 88149"
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета
statistics=["UnwReach", "Reach"]
# Указываем порядок разбики, 
# в оси media перечислим все уровни, которые хотим получить в виде иерархии
structure =  {
    "usetype": False,
    "media": ["holding", "site", "section", "subsection" ]
  }
# 
# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_hierarchy = rtask.result2hierarchy(rtask.get_result(task_audience), project_name=project_name)
df_hierarchy

Задача <395cf4a3-9216-443c-ade9-5e7affcdc54d> поступила в обработку
Расчет задачи [= = = = ] время расчета: 0:00:14.416924


,prj_name,media_holding,media_site,media_section,media_subsection,stat_unwreach,stat_reach
0,hierarchy,Google Sites,-,-,-,15554.0,50387.695053
3,hierarchy,Google Sites,Google (ru+com),-,-,14619.0,46880.730368
9,hierarchy,Google Sites,Google (ru+com),Google (ru+com),-,8577.0,23273.330615
16,hierarchy,Google Sites,Google (ru+com),Google (ru+com),Google (ru+com),8577.0,23273.330615
10,hierarchy,Google Sites,Google (ru+com),Google (ru+com) // Mobile app,-,11952.0,39199.844015
24,hierarchy,Google Sites,Google (ru+com),Google (ru+com) // Mobile app,Google (ru+com) // Mobile app,11952.0,39199.844015
6,hierarchy,Google Sites,Google (ru+com),mobile_hidden_88150,-,3111.0,10285.019790
18,hierarchy,Google Sites,Google (ru+com),mobile_hidden_88150,mobile_hidden_88150,3111.0,10285.019790
2,hierarchy,Google Sites,Google // Play,-,-,11351.0,36266.690680
5,hierarchy,Google Sites,Google // Play,Google // Play,-,1296.0,3085.917398


## Экспорт в Excel

In [4]:
writer = pd.ExcelWriter(rtask.get_excel_filename('hierarсhy'))
df_info = rtask.get_report_info()
df_hierarchy.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()